In [2]:
import numpy as np

def project_to_xy_plane(p1, p2):
    """
    如何理解这个函数？
    函数目的: 求出从p1到p2的直线与xy平面的交点

    为了方便讨论, 假设p1和p2在xoy平面的上方, 且p1在p2之上
    vect是一条从p1指向p2的向量
    (z2 / vect[2]) * vect也就是
    (z1 / (z2-z1)) * vect, 就是把vect向量延长z1/(z1-z2)倍, 且取反方向
    从几何角度来看, 延长后的向量的起点(因为取反)恰好在xoy平面上
    假设起点为p3
    那么延长后的向量(z1 / (z1-z2)) * vect = p1 - p3
    进而: p3 = p1 - (p1 - p3)
    """
    x1, y1, z1 = p1
    x2, y2, z2 = p2
    if z2 < z1:
        z2 = z1 + 1e-2  # TODO, bad hack
    vect = p2 - p1
    return p1 - (z2 / vect[2]) * vect
    

p1 = np.array([1, 1, 3])
p2 = np.array([1, 1, 2])
project_to_xy_plane(p1, p2)

array([ 1.  ,  1.  , -0.01])

In [4]:
import numpy as np

def project_to_xy_plane(p1, p2):
    """
    如何理解这个函数？
    函数目的: 求出从p1到p2的直线与xy平面的交点

    为了方便讨论, 假设p1和p2在xoy平面的上方, 且p1在p2之上
    vect是一条从p1指向p2的向量
    (z2 / vect[2]) * vect也就是
    (z1 / (z2-z1)) * vect, 就是把vect向量延长z1/(z1-z2)倍, 且取反方向
    从几何角度来看, 延长后的向量的起点(因为取反)恰好在xoy平面上
    假设起点为p3
    那么延长后的向量(z1 / (z1-z2)) * vect = p1 - p3
    进而: p3 = p1 - (p1 - p3)
    """
    x1, y1, z1 = p1
    x2, y2, z2 = p2
    vect = p2 - p1
    return p1 - (z1 / vect[2]) * vect
    
p1 = np.array([1, 1, 3])
p2 = np.array([1, 1, 2])
project_to_xy_plane(p1, p2)

array([1., 1., 0.])

In [5]:
import numpy as np

data = np.array([[1, 2, 3],
                 [4, 5, 6],
                 [7, 8, 9]])

def custom_function(arr):
    return np.sum(arr)

result_axis_0 = np.apply_along_axis(custom_function, axis=0, arr=data)
result_axis_1 = np.apply_along_axis(custom_function, axis=1, arr=data)

print("Result along axis 0:", result_axis_0)
print("Result along axis 1:", result_axis_1)


Result along axis 0: [12 15 18]
Result along axis 1: [ 6 15 24]


In [18]:
import numpy as np

# Define u_range and v_range
u_range = range(0, 10, 1)
v_range = range(0, 10, 1)

# Use nested list comprehension and conditional statement to generate the result
result = np.array([[[u, v] for v in v_range if u + v < 10] for u in u_range])

# Print the result
for row in result:
    print(row)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (10,) + inhomogeneous part.

In [19]:
import numpy as np

# Define u_range and v_range
u_range = range(0, 10, 1)
v_range = range(0, 10, 1)

# Use nested list comprehension and conditional statement to generate the result
result = [[[u, v] for v in v_range if u + v < 10] for u in u_range]

# Convert the result to a regular Python list
result = list(result)

# Print the result
for row in result:
    print(row)


[[0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [0, 6], [0, 7], [0, 8], [0, 9]]
[[1, 0], [1, 1], [1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [1, 7], [1, 8]]
[[2, 0], [2, 1], [2, 2], [2, 3], [2, 4], [2, 5], [2, 6], [2, 7]]
[[3, 0], [3, 1], [3, 2], [3, 3], [3, 4], [3, 5], [3, 6]]
[[4, 0], [4, 1], [4, 2], [4, 3], [4, 4], [4, 5]]
[[5, 0], [5, 1], [5, 2], [5, 3], [5, 4]]
[[6, 0], [6, 1], [6, 2], [6, 3]]
[[7, 0], [7, 1], [7, 2]]
[[8, 0], [8, 1]]
[[9, 0]]


In [11]:
u_range = list(range(0, 10, 1))
u_range

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]